In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.decomposition import PCA

from time import time

In [25]:
"""
By doing this file, we can finally seperate original dataset into test and train dataset 

"""


'\nBy doing this file, we can finally seperate original dataset into test and train dataset \n\n'

In [2]:
df = pd.read_csv('genelevel_rsem_expectedcounts_byrid_nooutliers.counts.matrix.tsv', sep='\t',index_col=0)
df.head() #(58051, 453)
#df.shape 


,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,90_2,91_2,92_3,93_2,94_4,95_4,96_3,97_2,98_3,99_1
ENSG00000000003,225.00,252.0,136.0,166.0,207.0,121.0,127.0,304.0,227.0,224.0,...,132.0,149.0,186.0,71.0,272.0,136.0,324.0,158.0,168.0,167.0
ENSG00000000005,0.00,0.0,1.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0
ENSG00000000419,254.00,301.0,173.0,264.0,307.0,140.0,164.0,279.0,216.0,274.0,...,369.0,148.0,265.0,86.0,326.0,283.0,300.0,242.0,286.0,207.0
ENSG00000000457,422.99,510.0,272.0,301.0,417.0,116.0,198.0,278.0,243.0,297.0,...,274.0,163.0,227.0,96.0,418.0,338.0,277.0,342.0,343.0,140.0
ENSG00000000460,272.00,310.0,204.0,224.0,227.0,149.0,253.0,171.0,274.0,198.0,...,280.0,168.0,308.0,86.0,284.0,260.0,179.0,210.0,225.0,124.0


In [3]:
anno = pd.read_csv('AnnotationFile.tsv', sep = '\t', index_col = [0])
anno.head()

,external_gene_name,chromosome_name,start_position,end_position,strand,gene_length,gene_biotype
ensembl_gene_id,,,,,,,
ENSG00000000003,TSPAN6,X,100627109,100639991,-1,4535.0,protein_coding
ENSG00000000005,TNMD,X,100584802,100599885,1,1610.0,protein_coding
ENSG00000000419,DPM1,20,50934867,50958555,-1,1207.0,protein_coding
ENSG00000000457,SCYL3,1,169849631,169894267,-1,6883.0,protein_coding
ENSG00000000460,C1orf112,1,169662007,169854080,1,5967.0,protein_coding


In [4]:
#Removing X&Y chromosome
anno = anno[anno.chromosome_name != 'X']
anno.head()
anno = anno[anno.chromosome_name != 'Y']
anno.head()

,external_gene_name,chromosome_name,start_position,end_position,strand,gene_length,gene_biotype
ensembl_gene_id,,,,,,,
ENSG00000000419,DPM1,20,50934867,50958555,-1,1207.0,protein_coding
ENSG00000000457,SCYL3,1,169849631,169894267,-1,6883.0,protein_coding
ENSG00000000460,C1orf112,1,169662007,169854080,1,5967.0,protein_coding
ENSG00000000938,FGR,1,27612064,27635277,-1,3474.0,protein_coding
ENSG00000000971,CFH,1,196651878,196747504,1,8145.0,protein_coding


In [5]:
both = df.join(anno)#
both= both.dropna(axis=0,how='any')
both.shape # (55169, 460)
both.head()


,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,97_2,98_3,99_1,external_gene_name,chromosome_name,start_position,end_position,strand,gene_length,gene_biotype
ENSG00000000419,254.00,301.00,173.00,264.00,307.0,140.00,164.00,279.00,216.00,274.00,...,242.0,286.00,207.0,DPM1,20,50934867.0,50958555.0,-1.0,1207.0,protein_coding
ENSG00000000457,422.99,510.00,272.00,301.00,417.0,116.00,198.00,278.00,243.00,297.00,...,342.0,343.00,140.0,SCYL3,1,169849631.0,169894267.0,-1.0,6883.0,protein_coding
ENSG00000000460,272.00,310.00,204.00,224.00,227.0,149.00,253.00,171.00,274.00,198.00,...,210.0,225.00,124.0,C1orf112,1,169662007.0,169854080.0,1.0,5967.0,protein_coding
ENSG00000000938,21.00,12.00,21.00,11.00,16.0,34.00,15.00,19.00,17.00,18.00,...,29.0,10.00,12.0,FGR,1,27612064.0,27635277.0,-1.0,3474.0,protein_coding
ENSG00000000971,165.00,178.96,111.49,181.77,244.0,230.67,196.97,264.92,179.72,163.48,...,174.0,211.66,162.7,CFH,1,196651878.0,196747504.0,1.0,8145.0,protein_coding


In [6]:
cleandf = both.drop(['external_gene_name','chromosome_name','start_position','end_position','strand','gene_length','gene_biotype'],axis = 1)
cleandf.head()

,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,90_2,91_2,92_3,93_2,94_4,95_4,96_3,97_2,98_3,99_1
ENSG00000000419,254.00,301.00,173.00,264.00,307.0,140.00,164.00,279.00,216.00,274.00,...,369.00,148.00,265.00,86.0,326.00,283.0,300.00,242.0,286.00,207.0
ENSG00000000457,422.99,510.00,272.00,301.00,417.0,116.00,198.00,278.00,243.00,297.00,...,274.00,163.00,227.00,96.0,418.00,338.0,277.00,342.0,343.00,140.0
ENSG00000000460,272.00,310.00,204.00,224.00,227.0,149.00,253.00,171.00,274.00,198.00,...,280.00,168.00,308.00,86.0,284.00,260.0,179.00,210.0,225.00,124.0
ENSG00000000938,21.00,12.00,21.00,11.00,16.0,34.00,15.00,19.00,17.00,18.00,...,21.00,14.00,9.00,21.0,17.00,11.0,17.00,29.0,10.00,12.0
ENSG00000000971,165.00,178.96,111.49,181.77,244.0,230.67,196.97,264.92,179.72,163.48,...,136.07,199.33,215.67,56.0,299.43,151.8,147.81,174.0,211.66,162.7


In [7]:
ndf = cleandf.T
ndf['MGS_LEVEL'] = 0
ndf.head()

,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,...,ENSG00000283688,ENSG00000283689,ENSG00000283690,ENSG00000283691,ENSG00000283693,ENSG00000283695,ENSG00000283696,ENSG00000283698,ENSG00000283699,MGS_LEVEL
100_2,254.0,422.99,272.0,21.0,165.00,107.0,449.0,757.0,787.0,2085.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0
101_3,301.0,510.00,310.0,12.0,178.96,150.0,624.0,872.0,618.0,1979.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0
102_2,173.0,272.00,204.0,21.0,111.49,94.0,219.0,643.0,372.0,1195.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0
103_3,264.0,301.00,224.0,11.0,181.77,78.0,583.0,698.0,465.0,1481.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0
104_2,307.0,417.00,227.0,16.0,244.00,114.0,499.0,627.0,540.0,1578.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0


In [8]:
new = pd.DataFrame()
new = ndf.reset_index()
new.head()

,index,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000283688,ENSG00000283689,ENSG00000283690,ENSG00000283691,ENSG00000283693,ENSG00000283695,ENSG00000283696,ENSG00000283698,ENSG00000283699,MGS_LEVEL
0,100_2,254.0,422.99,272.0,21.0,165.00,107.0,449.0,757.0,787.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0
1,101_3,301.0,510.00,310.0,12.0,178.96,150.0,624.0,872.0,618.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0
2,102_2,173.0,272.00,204.0,21.0,111.49,94.0,219.0,643.0,372.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0
3,103_3,264.0,301.00,224.0,11.0,181.77,78.0,583.0,698.0,465.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0
4,104_2,307.0,417.00,227.0,16.0,244.00,114.0,499.0,627.0,540.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0


In [9]:
mlist = new ['index'].apply(lambda x:x[-1]).tolist()
ndf['MGS_LEVEL'] = mlist
level = ndf.pop('MGS_LEVEL')
ndf.insert(0,'MGS_LEVEL',level)
ndf.head()

,MGS_LEVEL,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000283686,ENSG00000283688,ENSG00000283689,ENSG00000283690,ENSG00000283691,ENSG00000283693,ENSG00000283695,ENSG00000283696,ENSG00000283698,ENSG00000283699
100_2,2,254.0,422.99,272.0,21.0,165.00,107.0,449.0,757.0,787.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
101_3,3,301.0,510.00,310.0,12.0,178.96,150.0,624.0,872.0,618.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
102_2,2,173.0,272.00,204.0,21.0,111.49,94.0,219.0,643.0,372.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
103_3,3,264.0,301.00,224.0,11.0,181.77,78.0,583.0,698.0,465.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
104_2,2,307.0,417.00,227.0,16.0,244.00,114.0,499.0,627.0,540.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0


In [10]:
from sklearn.model_selection import train_test_split

#split dataset into train and test dataset
X, y = ndf.iloc[:,1:], ndf.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

#shape of train and test
X_train.shape, y_train.shape # ((407, 55169), (407,))
X_test.shape, y_test.shape #((46, 55169), (46,))

((46, 55169), (46,))

In [11]:
X_test.head()

,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,...,ENSG00000283686,ENSG00000283688,ENSG00000283689,ENSG00000283690,ENSG00000283691,ENSG00000283693,ENSG00000283695,ENSG00000283696,ENSG00000283698,ENSG00000283699
233_2,218.0,296.0,139.0,16.0,127.00,87.00,279.0,419.0,441.0,1356.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
158_3,279.0,359.0,376.0,25.0,211.00,123.00,489.0,720.0,459.0,1619.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
419_4,258.0,320.0,445.0,43.0,280.55,140.98,411.0,887.0,664.0,1992.97,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
430_3,308.0,504.0,382.0,13.0,335.00,235.99,926.0,811.0,370.0,1567.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0
350_1,298.0,396.0,308.0,51.0,399.22,118.00,441.0,1244.0,681.0,2412.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0


In [12]:
one = X_train[X_train.index.str.contains('_4')]
one.shape # 97 / 160 / 98 / 52

(52, 55169)

In [13]:
stage4 = X_test[X_test.index.str.contains('_4')]
#stage1- 8
#stage2 - 15 
#stage3 -14
#stage4 - 9 

In [14]:
result = X_train.T
result.head()

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,202.0,243.00,230.0,270.0,232.00,231.00,174.00,164.00,244.0,207.0,...,257.00,274.00,230.00,327.00,252.0,285.0,237.00,223.0,278.00,159.0
ENSG00000000457,261.0,369.00,355.0,375.0,317.00,273.00,254.00,198.00,229.0,344.0,...,335.00,297.00,348.00,389.00,461.0,272.0,419.00,376.0,312.00,380.0
ENSG00000000460,194.0,188.00,234.0,236.0,385.00,154.00,144.00,253.00,218.0,214.0,...,331.00,198.00,192.00,274.00,251.0,322.0,364.00,129.0,322.00,252.0
ENSG00000000938,25.0,15.00,14.0,14.0,14.00,14.00,10.00,15.00,10.0,20.0,...,20.00,18.00,43.00,85.00,13.0,8.0,17.00,14.0,23.00,2.0
ENSG00000000971,428.0,91.06,83.0,184.0,167.78,138.79,105.76,196.97,236.0,61.0,...,118.77,163.48,200.57,266.55,175.0,344.0,146.84,67.0,367.51,97.0


In [22]:
result.to_csv("./train_genelevel_90%.tsv", sep='\t')

In [23]:
result2= X_test.T
result2.shape

(55169, 46)

In [24]:
result2.to_csv("./test_genelevel_10%.tsv", sep='\t')